In [2]:
! pip install pandas


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Tanu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd

# Load Excel file
df = pd.read_excel("flipr_data.xlsx")

# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Fill NaNs with 0 for math operations
df['Credit (INR)'] = df['Credit (INR)'].fillna(0)
df['Debit (INR)'] = df['Debit (INR)'].fillna(0)

# Sort by date just in case
df = df.sort_values(by='Date')


In [16]:
import pandas as pd
import json

# ---------------------------
# Load Data
# ---------------------------
def load_data(transaction_file, merchant_file):
    # Load transaction data
    transactions = pd.read_excel(transaction_file)
    transactions['Date'] = pd.to_datetime(transactions['Date'])
    
    # Load merchant data
    with open(merchant_file, 'r') as file:
        merchants = json.load(file)
    
    return transactions, merchants

transactions, merchants = load_data('flipr_data.xlsx', 'merchant_data.json')

# ---------------------------
# Extract Categories from Merchant Data
# ---------------------------
def extract_categories(merchants):
    # Extract unique categories from merchant data
    categories = list({merchant['category'] for merchant in merchants.values()})
    return categories

categories = extract_categories(merchants)

# ---------------------------
# Simulate Asking for Amounts (Fallback for Non-Interactive Environments)
# ---------------------------
def collect_category_amounts(categories):
    category_amounts = {}
    
    print("Please enter amounts for each category:")
    for category in categories:
        while True:
            try:
                # Simulate user input with predefined values (replace with real input if possible)
                print(f"Simulating input for {category}...")
                amount = float(input(f"Enter amount for {category}: "))  # Replace this with predefined values if needed
                category_amounts[category] = round(amount, 2)
                break
            except ValueError:
                print("Invalid input. Please enter a numeric value.")
    
    return category_amounts

try:
    # Collect amounts interactively or simulate input
    category_amounts = collect_category_amounts(categories)
except Exception as e:
    print(f"Interactive input not supported: {e}")
    # Simulated inputs (replace these with actual user inputs in a real environment)
    category_amounts = {category: 1000.0 for category in categories}  # Example: Assigning ₹1000 to each category

print("\nCollected Category Amounts:")
for category, amount in category_amounts.items():
    print(f"{category}: ₹{amount}")

# ---------------------------
# Integrate Merchant Data into Transactions
# ---------------------------
def integrate_merchant_data(transactions, merchants):
    merchant_map = {k.strip().lower(): v for k, v in merchants.items()}
    
    transactions['merchant_info'] = (
        transactions['Description']
        .str.strip()
        .str.lower()
        .map(merchant_map)
        .apply(lambda x: x if isinstance(x, dict) else None)
    )
    
    return transactions

transactions = integrate_merchant_data(transactions, merchants)

# ---------------------------
# Generate Financial Report
# ---------------------------
def generate_financial_report(transactions):
    report = {
        'total_income': transactions['Credit (INR)'].sum(),
        'total_expenses': transactions['Debit (INR)'].sum(),
        'current_balance': transactions['Balance (INR)'].iloc[-1],
        'category_spending': transactions.groupby('Category')['Debit (INR)'].sum().to_dict()
    }
    return report

financial_report = generate_financial_report(transactions)

print("\nFinancial Report:")
for key, value in financial_report.items():
    if isinstance(value, dict):
        print(f"{key}:")
        for sub_key, sub_value in value.items():
            print(f"  {sub_key}: ₹{sub_value}")
    else:
        print(f"{key}: ₹{value}")

# ---------------------------
# Generate SMART Goals
# ---------------------------
def generate_exciting_goals(report):
    goals = []

    # Get top spending category with proper sorting
    top_category, top_amount = max(
        report['category_spending'].items(), 
        key=lambda x: x[1]
    )

    # Goal 1: Top Category Challenge
    goals.append({
        'type': 'Spending Champion',
        'icon': '🏆',
        'title': f"Master of {top_category}!",
        'message': f"You spent ₹{top_amount:,.2f} on {top_category} - that's {top_amount/report['total_expenses']:.1%} of total expenses!",
        'challenge': f"Reduce this by 15% to save ₹{top_amount*0.15:,.2f}/month",
        'reward': "Unlock premium budgeting tools!"
    })

    # Goal 2: Savings Boost with auto-adjust
    savings_target = min(0.25, max(0.15, (report['total_income'] - report['total_expenses'])/report['total_income']))
    goals.append({
        'type': 'Savings Adventure',
        'icon': '💰',
        'title': "Wealth Building Challenge!",
        'message': f"Potential monthly savings: ₹{report['total_income']*savings_target:,.2f}",
        'target': f"Save {savings_target:.0%} of income",
        'progress': f"Current: ₹{report['total_income'] - report['total_expenses']:,.2f}",
        'badge': "Financial Freedom Champion"
    })

    return goals

exciting_goals = generate_exciting_goals(financial_report)

print("\nSMART Goals:")
for goal in exciting_goals:
    print(f"- {goal['title']}: {goal['message']}")

# ---------------------------
# Adaptive Budgeting System
# ---------------------------
def create_playful_budget(report):
    budget = {}
    
    emoji_map = {
        'Food': '🍔', 
        'Travel': '✈️', 
        'Shopping': '🛍️',
        'Entertainment': '🎮',
        'Utilities': '💡',
        'Healthcare': '🏥'
    }

    for category, amount in report['category_spending'].items():
        base_category = category.split()[0]
        icon = emoji_map.get(base_category, "✨")
        
        budget[f"{icon} {category}"] = {
            "allocated": amount * 1.1,
            "challenge": f"Stay under ₹{amount * 1.15:,.2f}",
            "tips": [
                f"Potential monthly save: ₹{amount * 0.1:,.2f}",
                "Weekly spending alerts available"
            ]
        }

    budget["💰 Smart Savings"] = {
        "target": report["total_income"] * 0.2,
        "current": report["total_income"] - report["total_expenses"],
        "milestones": {
            "25%": "Financial health report",
            "50%": "Investment guide access",
            "100%": "Celebration reward!"
        }
    }

    return budget

playful_budget = create_playful_budget(financial_report)

print("\nRecommended Budget:")
for category, details in playful_budget.items():
    print(f"{category}:")
    for key, value in details.items():
        print(f"  {key}: {value}")

# ---------------------------
# Final Output Integration
# ---------------------------
final_output = {
    "financial_summary": financial_report,
    "smart_goals": exciting_goals,
    "recommended_budget": playful_budget,
}

print("\nFinal Output:")
print(final_output)


Please enter amounts for each category:
Simulating input for Fast Food...
Simulating input for movie...

Collected Category Amounts:
Fast Food: ₹1000000.0
movie: ₹20000000.0

Financial Report:
total_income: ₹5322099.99
total_expenses: ₹2264816.4000000004
current_balance: ₹3107283.59
category_spending:
  Entertainment: ₹238658.86
  Food & Dining: ₹314545.99
  Fuel: ₹240983.28
  Healthcare: ₹300268.39
  Income: ₹0.0
  Others: ₹259271.28
  Shopping: ₹221646.38999999998
  Subscriptions: ₹165349.63999999998
  Travel: ₹221031.07
  Utilities: ₹303061.5

SMART Goals:
- Master of Food & Dining!: You spent ₹314,545.99 on Food & Dining - that's 13.9% of total expenses!
- Wealth Building Challenge!: Potential monthly savings: ₹1,330,525.00

Recommended Budget:
🎮 Entertainment:
  allocated: 262524.746
  challenge: Stay under ₹274,457.69
  tips: ['Potential monthly save: ₹23,865.89', 'Weekly spending alerts available']
🍔 Food & Dining:
  allocated: 346000.58900000004
  challenge: Stay under ₹361,727

Enter new expenses for today (₹):

💰 Financial Summary:
Current Balance: ₹3,105,283.59
Total Monthly Expenses: ₹2,267,816.40
New Expenses Added: 2 transactions

📊 Category Breakdown:
Entertainment       : ₹238,658.86
Fast Food           : ₹1,000.00
Food & Dining       : ₹314,545.99
Fuel                : ₹240,983.28
Healthcare          : ₹300,268.39
Income              : ₹0.00
Others              : ₹259,271.28
Shopping            : ₹221,646.39
Subscriptions       : ₹165,349.64
Travel              : ₹221,031.07
Utilities           : ₹303,061.50
movie               : ₹2,000.00

📅 Today's Spending: ₹3,000.00
   Status: Under Budget
